In [ ]:
import numpy as np
import pylab as plt
import matplotlib
import datetime
%matplotlib inline

### Modelling Intraday Volatility

http://pure.au.dk/portal/files/116799978/rp17_30.pdf

In [ ]:
# Binural Process, pg 22
N = 8 * 3600
A = 0.75
B = 0.25
C = 0.088929198
a1  = a2 = 10
t = np.arange(N)/N
sigma = lambda t: C + A * np.exp(-a1 * t) + B * np.exp(-a2 * (1-t))
plt.plot(sigma(t),'b-',lw=5)
plt.xlabel('time');
plt.ylabel('annualized intraday volatility');

### Modelling Intraday Spread

https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=6&cad=rja&uact=8&ved=2ahUKEwjnjLH2vd7kAhVEILcAHQqxAr0QFjAFegQIABAB&url=https%3A%2F%2Fwww.researchgate.net%2Fpublication%2F228318832_A_Simple_Approximation_of_Intraday_Spreads_Using_Daily_Data&usg=AOvVaw2DmrwFF9qBq3AeEaMcSblY

https://www3.nd.edu/~scorwin/papers/high-low_spreads_Internet%20Appendix.pdf

In [ ]:
N = 8 * 3600
At = 0.00075
Bt = 0.00015
Ct = 0.06929198
a1t  = a2t = 7
t = np.arange(N)/N
spread = lambda t: Ct + At * np.exp(-a1t * t) + Bt * np.exp(-a2t * (1-t))
plt.plot(spread(t),'r-',lw=5)
plt.xlabel('time');
plt.ylabel('bid/ask spread');

### Calculating Bid/Ask

In [ ]:
def make_price(P0,vol_multiplier):
    '''
    vol_multilplier: simple 'fudge factor' for changing volatility
    P0: starting price
    '''
    np.random.seed(271)
    ask = []
    bid = []
    Pmid = P0
    for i in t:
        Pmid += random.randn() * sigma(i)/sqrt(252*8*3600) * vol_multiplier
        bas = spread(i)
        bid.append(Pmid - bas/2.)
        ask.append(Pmid + bas/2.)
    return bid,ask 

bid,ask = make_price(10,10)    

In [ ]:
plt.plot(bid,label='bid');
plt.plot(ask,label='ask');
plt.legend(loc='best');
plt.xlabel('time in secs');
plt.ylabel('price');
plt.show()


plt.plot(np.array(ask)-np.array(bid),label='spread')
plt.legend()
plt.xlabel('time in secs');
plt.ylabel('spread');

### Trading Cost Variation

In [ ]:
def calc_exec_cost(T,n,bid,ask):

    bid0 = bid[T]
    ask0 = ask[T]
    mid0 = (ask0 + bid0)/2.

    bid1 = bid[T+n]
    ask1 = ask[T+n]
    mid1 = (ask1 + bid1)/2.

    cost = ask1 - mid0
    return cost

calc_exec_cost(100,2,bid,ask)

In [ ]:
inc0 = 10
C1 = []
fast_exec_period = 2
for T in range(0,N,inc0):
    C1.append(calc_exec_cost(T,fast_exec_period,bid,ask))

C2 = []
slow_exec_period = 8
for T in range(0,N,inc0):
    C2.append(calc_exec_cost(T,slow_exec_period,bid,ask))
    
plt.plot(C2,label='slow execution');
plt.plot(C1,label='fast execution');
plt.legend(loc='best');
plt.xlabel('time in secs * %s'%inc0);
plt.ylabel('trading cost');
plt.title('Time variation in execution cost')

### Analysis

In [ ]:
plt.hist(C1[:600],80,label='open');
plt.hist(C1[1200:1800],40,label='midday');
plt.legend()
plt.title('Trading cost distribution at different parts of the day');

In [ ]:
inc = 100
Mid = []
for i in range(inc,N,inc):
    Mid.append(np.mean(C1[i-inc:i]))

plt.plot(Mid)
plt.legend(loc='best')
plt.xlabel('time in secs*%s'%inc)
plt.ylabel('cost')
plt.title('execution cost averaged over %s periods'%inc)
plt.legend()
plt.grid()
plt.show()

### Conclusions

- Execution time matters
- Execution cost is, on average, lower mid-day (but so is liquidity)
- Difference is more pronounced in low-vol conditions